In [356]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 9
year = 2022

data = get_data(day=day, year=year)

# submit(cube.sum(), part="a", day=day, year=year)

# data

In [303]:
data = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""

In [371]:

def print_grid(**kwargs):
    for j in range(-10, 10):
        for i in range(-20, 20):
            # print(i,j)
            c = '.'
            for l,pos in kwargs.items():
                if pos == (i,-j):
                    c = l
            print(c, end='')
        print("")
    print("")

In [406]:
dirs = {'R': (1, 0), 'U': (0, 1), 'L': (-1, 0), 'D': (0, -1)}
move = lambda X,d: (X[0]+d[0], X[1]+d[1])

def solve(data, num_knots = 10):
    K = [(0,0)] * num_knots
    visited = set(K)

    for l in data.split("\n"):
        dir,steps = l.split()
        for _ in range(int(steps)):
            K[0] = move(K[0], dirs[dir])
            for k in range(1, len(K)):
                delta = [K[k-1][i] - K[k][i] for i in (0,1)]
                if abs(max(delta, key=abs)) > 1:
                    K[k] = move(K[k], [x//(abs(x) or 1) for x in delta])
            visited.add(K[-1])
            
    return len(visited)

solve(data, 2), solve(data, 10)

(6243, 2630)

(6243, 2630)

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0]])

['D', 'C']